In [1]:
import pandas
import torch
import torch.nn as nn
import torch.nn.functional as Fu
import numpy as np
from torch.autograd import Variable

In [2]:
data=np.loadtxt(open("extended.csv", "rb"), delimiter=",", skiprows=0, dtype=float)
print(data.shape)

(4368, 19)


In [3]:
real_data = torch.Tensor()
real_data = Variable(torch.from_numpy(data))
print(real_data)

Variable containing:
 0.0800  0.0800  0.2800  ...  -0.2000 -0.0400  0.0800
 0.1600  0.1600  0.0800  ...  -0.2000 -0.0400  0.0800
 0.4000  0.4000  0.4000  ...  -0.2000 -0.0400  0.0800
          ...             ⋱             ...          
 0.9600  0.9600  0.8400  ...   0.3600  0.5200  0.6400
-1.0000 -1.0000  0.8800  ...   0.4000  0.5600  0.6800
-0.9600 -0.9600  0.9200  ...   0.4400  0.6000  0.7200
[torch.DoubleTensor of size 4368x19]



In [46]:
import numpy.random
#returns 50 inputs for the Generator (3 features chord and 1 noise)
def generator_input(n):
    ret = Variable(torch.cuda.FloatTensor(n,4))
    tune = torch.rand(n)*2 -1
    tune2 = tune + 4/25
    tune3 = tune + 7/25
    ret[:,0]=tune
    ret[:,1]=tune2
    ret[:,2]=tune3
    ret[:,3]=torch.rand(n)*2 -1
    return ret

In [43]:
class Discri(nn.Module):
    def __init__(self):
        super(Discri, self).__init__()
        self.lin1 = nn.Linear(19, 19)
        self.lin2 = nn.Linear(19, 19)
        self.lin3 = nn.Linear(19, 1)
 
    def forward(self, x):
        x = Fu.leaky_relu(self.lin1(x))
        x = Fu.leaky_relu(self.lin2(x))
        return Fu.sigmoid(self.lin3(x))

In [6]:

class Gene(nn.Module):
    def __init__(self):
        super(Gene, self).__init__()
        self.lin1 = nn.Linear(4, 4)
        self.lin1.weights = torch.ones(4,4)
        self.lin2 = nn.Linear(4,16)
        self.lin2.weights = torch.ones(4,16)
        self.lin3 = nn.Linear(16, 16)
        self.lin3.weights = torch.ones(16,16)
        
    def forward(self, x):
        x = Fu.leaky_relu(self.lin1(x))
        x = Fu.dropout(Fu.leaky_relu(self.lin2(x)))
        return Fu.leaky_relu(self.lin3(x))

In [47]:
#generates a set of 4000 samples of data generated by the GENERATOR with random chords
def gene_fake(n):

    ret = Variable(torch.cuda.FloatTensor(n,19))
    gene_input = generator_input(n)
    ret[:,:16] = gene(gene_input)
    ret[:,16:] = gene_input[:,:3]

    return ret

In [49]:
#initializations 

discri = Discri()
gene = Gene()
discri.cuda()
gene.cuda()
learning_rate = 1e-4
loss_disc = torch.nn.MSELoss()
loss_gene = torch.nn.MSELoss()
disc_optim = torch.optim.SGD(discri.parameters(), lr=learning_rate)
gene_optim = torch.optim.Adam(gene.parameters(), lr=learning_rate)

input = Variable(torch.cuda.FloatTensor(8368,19))
input[:4368] = real_data
input[4368:]= gene_fake(4000)

exp_output = Variable(torch.cuda.FloatTensor(8368))
exp_output[:4368] = torch.rand(4368)*0.4 + 0.8 #for real data
exp_output[4368:] = torch.rand(4000)*0.3 #dor fake data

batch_size = 50

In [55]:
for epoch in range(0,20):
    #training of discriminator
    for i in range(0,500):
        batch_index = (batch_size*i)%len(input)
        decision = discri(input[batch_index : (batch_index + batch_size)])
        loss = loss_disc(decision, exp_output[batch_index : (batch_index + batch_size)])      
        disc_optim.zero_grad()
        loss.backward(retain_graph=True)
        disc_optim.step()

    print("epoch :",epoch)
    print("discrimator loss :",loss.data[0])

    #training og generator
    for i in range(0,500):
        generated = gene_fake(2000)
        result = discri(generated)
        loss = loss_gene(result,Variable(torch.ones(2000)).cuda())
        gene_optim.zero_grad()
        loss.backward(retain_graph=True)
        gene_optim.step()
    print("generator loss :",loss.data[0])


epoch : 0
discrimator loss : 0.12481730431318283
generator loss : 0.24796412885189056
epoch : 1
discrimator loss : 0.12543660402297974
generator loss : 0.23510096967220306
epoch : 2
discrimator loss : 0.12605077028274536
generator loss : 0.22226569056510925
epoch : 3
discrimator loss : 0.12665966153144836
generator loss : 0.20553824305534363
epoch : 4
discrimator loss : 0.1272633969783783
generator loss : 0.18327723443508148
epoch : 5
discrimator loss : 0.12786227464675903
generator loss : 0.15729926526546478
epoch : 6
discrimator loss : 0.12845616042613983
generator loss : 0.1280916929244995
epoch : 7
discrimator loss : 0.1290450543165207
generator loss : 0.09896301478147507
epoch : 8
discrimator loss : 0.12962864339351654
generator loss : 0.07302025705575943
epoch : 9
discrimator loss : 0.13020657002925873
generator loss : 0.053199734538793564
epoch : 10
discrimator loss : 0.1307799369096756
generator loss : 0.036446716636419296
epoch : 11
discrimator loss : 0.13134849071502686
gener

In [56]:
list(gene.lin1.parameters())

[Parameter containing:
 -1.2350  0.6583  0.9945  0.1051
 -1.0299 -0.4995 -0.7366 -0.7646
  1.1266  1.0440  0.8862 -0.1970
 -0.9579 -1.2038 -0.0663  0.8526
 [torch.cuda.FloatTensor of size 4x4 (GPU 0)], Parameter containing:
  1.3984
  0.8881
  0.5217
  1.1358
 [torch.cuda.FloatTensor of size 4 (GPU 0)]]

In [58]:
#generate blues song data CCCC FFCC GGCC
C = 60.0
G = 67.0
F = 65.0
ret = Variable(torch.cuda.FloatTensor(12,4))
tune = torch.from_numpy(np.array([C,C,C,C,F,F,C,C,G,G,C,C]))
tune2 = tune + 4
tune3 = tune + 7
ret[:,0]=tune
ret[:,1]=tune2
ret[:,2]=tune3
ret[:,3]=torch.randperm(12)
data = gene(ret).round()
print(data)

Variable containing:

Columns 0 to 12 
 2066  1748   280  1983  2291   -12  2071   -10   473    -1   -12    -7  1304
 2070  1752   281  1987  2296   -12  2075   -10   474    -1   -12    -7  1306
 2063  1746   280  1980  2288   -12  2068   -10   472    -1   -12    -7  1302
 2061  1744   280  1978  2286   -12  2066   -10   472    -1   -12    -7  1301
 2219  1878   301  2130  2462   -13  2225   -11   508    -1   -13    -8  1401
 2224  1882   302  2135  2467   -13  2229   -11   509    -1   -13    -8  1403
 2060  1743   280  1977  2284   -12  2065   -10   471    -1   -12    -7  1300
 2071  1753   281  1988  2297   -12  2076   -10   474    -1   -12    -7  1307
 2292  1940   311  2201  2543   -13  2298   -11   525    -1   -13    -8  1446
 2284  1934   310  2193  2534   -13  2290   -11   523    -1   -13    -8  1442
 2068  1750   281  1985  2293   -12  2073   -10   473    -1   -12    -7  1305
 2069  1751   281  1986  2295   -12  2074   -10   473    -1   -12    -7  1306

Columns 13 to 15 
  863 

In [59]:
#convert to midi
from midiutil.MidiFile import MIDIFile

mf = MIDIFile(2)    

# the melody track
track = 0   
time = 0
mf.addTrackName(track, time, "Melody")
mf.addTempo(track, time, 120)
channel = 0
volume = 100

for bar in data:
    for note in bar:
        if note.data[0] >0:
            mf.addNote(track, channel, note.data[0]+40, time, 0.25, volume)
        time = time +0.25

#the chord track
track = 1
time = 0
volume = 50
mf.addTrackName(track,time,"Chords")    
mf.addTempo(track, time, 120)

C = 60
E = 64
G = 67
F = 65
A = 69
B = 71

for i in range(0,4):
    mf.addNote(track,channel,C,time,4,volume)
    mf.addNote(track,channel,E,time,4,volume)
    mf.addNote(track,channel,G,time,4,volume)
    time=time+4
for i in range(0,2):
    mf.addNote(track,channel,F,time,4,volume)
    mf.addNote(track,channel,A,time,4,volume)
    mf.addNote(track,channel,C,time,4,volume)
    time=time+4
for i in range(0,2): 
    mf.addNote(track,channel,C,time,4,volume)
    mf.addNote(track,channel,E,time,4,volume)
    mf.addNote(track,channel,G,time,4,volume)  
    time=time+4
for i in range(0,2): 
    mf.addNote(track,channel,G,time,4,volume)
    mf.addNote(track,channel,B,time,4,volume)
    mf.addNote(track,channel,E,time,4,volume)
    time=time+4
for i in range(0,2): 
    mf.addNote(track,channel,C,time,4,volume)
    mf.addNote(track,channel,E,time,4,volume)
    mf.addNote(track,channel,G,time,4,volume)  
    time=time+4

with open("awesome.mid", 'wb') as outf:
    mf.writeFile(outf)

ModuleNotFoundError: No module named 'midiutil'